# Data Collection & Preprocessing 

The code in this notebook is for the use of scaping the latest comment titles and body from a specific subreddit, converting to a dataframe, and exporting to a .csv file


### Import Libaries

In [1]:
import pandas as pd
import praw
import sensitive as sens
from tqdm.notebook import tqdm

### Reddit API Scrape

To use the below code and function you must create a reddit account [here](https://www.reddit.com/) and register for use of the API. The username and password will be from your general Reddit account while your client id, cleint secret, and user agent will be from the API token your create attached to the account.

In [2]:
# enter your personal account info in accordingly
reddit = praw.Reddit(
    client_id=sens.client_id,
    client_secret=sens.client_secret,
    password=sens.password, 
    user_agent=sens.user_agent,
    username=sens.username
)

The below function takes in the above-created praw instance, the subreddit you would like to scape as a string, and the number of latest posts you would like to collect as an integer. 

The output will print the number of comments successfully scaped and a dataframe with each row including the comments category, body, and title.

In [3]:
def reddit_scraper(praw_object, sub_reddit, num_posts):
    # create submissions object to iterate over
    submissions = praw_object.subreddit(sub_reddit).hot(limit = num_posts)
    
    # create list of dictionaries for easy conversion to df
    posts = []
    comments = []
    for i, post in enumerate(submissions):
        comms = post.comments.list()
        posts.append({
            'categroy': post.subreddit,
            'title': post.title,
            'body': post.selftext,
            'sub': sub_reddit,
            'sub_post_id': i,
            'n_comments': len(comms)
        })
        for com in comms:
            comments.append({
                'sub_post_id': i,
                'text': com.body,
                'likes': com.likes,
            })
    
    # show number of articles collected and out df
    print(f'You collected {len(posts)} reddit comments about {sub_reddit}')
    return pd.DataFrame(posts), pd.DataFrame(comments)

In [4]:
subs = ['datascience', 'datasciencejobs', 'machinelearning', 
        'machinelearningjobs', 'learnmachinelearning',
        'learndatascience']
scraped = {}
for sub in tqdm(subs):
    scraped[sub] = reddit_scraper(reddit, sub, 2)

# # scape 1,000 articles on history and consipracy to model
# history_df = reddit_scapper(reddit, 'epicseven', 1_000)
# conspiracy_df = reddit_scapper(reddit, 'lewdseven', 1_000)

You collected 2 reddit comments about datascience
You collected 2 reddit comments about datasciencejobs
You collected 2 reddit comments about machinelearning
You collected 2 reddit comments about machinelearningjobs
You collected 2 reddit comments about learnmachinelearning
You collected 2 reddit comments about learndatascience



In [5]:
posts_l = []
comms_l = []
for _, tup in scraped.items():
    posts_l.append(tup[0])
    comms_l.append(tup[1])

In [6]:
posts_df = pd.concat(posts_l)
comms_df = pd.concat(comms_l)

In [7]:
posts_df

,categroy,title,body,sub,sub_post_id,n_comments
0,datascience,Weekly Entering & Transitioning Thread | 08 Ma...,_Bleep Bloop_. Welcome to this week's entering...,datascience,0,86
1,datascience,Open COVID-19 Dataset,I was frustrated with the maintenance issues i...,datascience,1,9
0,DataScienceJobs,[Hiring] Data Scientist/Senior Data Scientist ...,,datasciencejobs,0,0
1,DataScienceJobs,I am hopeless in my job search. Would you mind...,&#x200B;\n\nA little background about me - I a...,datasciencejobs,1,10
0,MachineLearning,[D] Machine Learning - WAYR (What Are You Read...,This is a place to share machine learning rese...,machinelearning,0,17
1,MachineLearning,[D] Advanced courses update,We have a [PhD level or Advanced courses](http...,machinelearning,1,75
0,MachineLearningJobs,[HIRING][REMOTE]Coding Specialist III at Navie...,What kind of candidates do we need?\n\n **Tool...,machinelearningjobs,0,0
1,MachineLearningJobs,[HIRING][REMOTE]Support Engineer,What kind of candidates do we need?\n\n **Tool...,machinelearningjobs,1,0
0,learnmachinelearning,I want to form a 3 to 4 person team of ML lear...,**SECOND EDIT**\n\nLol. Currently overwhelmed ...,learnmachinelearning,0,41
1,learnmachinelearning,A Geometric Intuition to Dimensionality Reduct...,,learnmachinelearning,1,1


In [8]:
comms_df

,sub_post_id,text,likes
0,0,Learn SQL\n\nI’m always aghast at how many que...,None
1,0,"I'm a high school student(10th grade, 16) who'...",None
2,0,Tomorrow I am having a coffee with a Senior Da...,None
3,0,What salary range to give to a fortune 500 com...,None
4,0,Upcoming (this May) graduate with two BS degre...,None
...,...,...,...
37,0,NLP is great. I do chatbots and stuff as a ML ...,None
38,0,Interested as well !,None
39,0,NLP is so fun. That's what I do for work. Let ...,None
40,0,Thanks.. can you suggest me some good resource...,None


In [9]:
posts_df.to_csv('posts.csv')
comms_df.to_csv('comms.csv')